In [2]:
import pandas as pd
import numpy as np
from itertools import chain

In [18]:
df_vote = pd.read_csv('H117_votes.csv')


In [43]:
#filter state by node file with only nodes needed
place_node = pd.read_csv('node district filtered state.csv')
# Merge df1 and df2 on 'icprs' column
merged_df_vote = pd.merge(df_vote, place_node[['icpsr']], on='icpsr', how='inner')

In [44]:
# distinct number of icpsr 457, 996 roll
icpsr = df_vote.icpsr.unique()
print(len(icpsr))
icpsr = merged_df_vote.icpsr.unique()
print(len(icpsr))
df_vote = merged_df_vote

69
68


In [45]:
# reshape df
re_df = df_vote.pivot(index='icpsr', columns='rollnumber', values='cast_code')

In [5]:
re_df.to_csv('widevote10_29.csv')

In [16]:
# re_df1 = re_df.astype('Int64')

In [17]:
# print(re_df1.iloc[1])

rollnumber
1      6
2      1
3      6
4      6
5      6
      ..
992    1
993    1
994    6
995    1
996    6
Name: 14854.0, Length: 996, dtype: Int64


In [46]:
print(len(re_df))

68


In [47]:
edgelist = []
for index, row in re_df.iterrows():
    # print (int(index))
    ex_df = re_df[index:]
    ex_df = ex_df.iloc[1:, :]
    # print(ex_df.shape)
    nrow = row.to_numpy()
    for exi, exr in ex_df.iterrows():
        nexr = exr.to_numpy()
        equal_pos = np.where(nrow == nexr)
        if(len(equal_pos[0]) > 450 ):
            edgelist.append([index,exi])

print(len(edgelist))

1279


In [48]:
# remove value not in the district node file 
newedgelist = list(chain.from_iterable(edgelist))
ic = df_vote.icpsr.unique()
differ = list(set(newedgelist)- set(ic) )
print(differ)
filter_edgelist = []
for edge in edgelist:
    if any(x in differ for x in edge):
        continue
    else: 
        filter_edgelist.append(edge)

print(len(edgelist))
print(len(filter_edgelist))
print(set(chain.from_iterable(filter_edgelist))- set(ic))

[]
1279
1279
set()


In [42]:
result_df = pd.DataFrame(filter_edgelist, columns =['Source', 'Target'])
result_df = result_df.astype('Int64')
result_df.to_csv('vote_edge_filter450.csv', index=False)

In [56]:
# tabel join distrct with icpsr
df_icpsr = pd.read_csv('H117_members.csv')
df_district = pd.read_csv('USA_117th_Congressional_Districts\congress district point.csv')

In [44]:
print(df_district.shape)

(436, 14)


In [61]:
for di, dis in df_district.iterrows():
    for mi, member in df_icpsr.iterrows():
        if (member['state_abbrev'] == dis.STATE_ABBR and member.district_code == dis.CDFIPS):
            df_district.loc[di].icpsr = member.icpsr
            break 



In [73]:
new_df = pd.merge(df_district, df_icpsr,  how='inner', left_on=['STATE_ABBR','CDFIPS'], right_on = ['state_abbrev','district_code'])


In [79]:
newdf1 = df_district.merge(df_icpsr, how='left', left_on=['STATE_ABBR','CDFIPS'], right_on = ['state_abbrev','district_code'])

In [81]:
newnew = newdf1.drop_duplicates(subset=['STATE_ABBR', 'CDFIPS'], keep="last")
print(newnew.shape)

(436, 36)


In [82]:
newnew.to_csv('node district.csv')

In [4]:
#test code :
# Sample data for df1
data1 = {'icprs': [1, 2, 3], 'state name': ['A', 'B', 'C']}
df1 = pd.DataFrame(data1)

# Sample data for df2
data2 = {'icprs': [2, 3, 4, 5, 3], 'other_column': ['X', 'Y', 'Z', 'W','S']}
df2 = pd.DataFrame(data2)

# Merge df1 and df2 on 'icprs' column
merged_df = pd.merge(df2, df1[['icprs']], on='icprs', how='inner')

# Now, merged_df contains only the rows from df2 where 'icprs' is present in df1
# If you want to keep all columns from df2, you can drop the 'icprs' column from df1
# and then merge on the 'icprs' column

# If you want to modify df2 in place, you can do:
# df2 = pd.merge(df2, df1[['icprs']], on='icprs', how='inner')

print(merged_df)

   icprs other_column
0      2            X
1      3            Y
2      3            S
